In [1]:
from cogent3 import load_aligned_seqs
from cogent3.draw.drawable import Drawable, AnnotatedDrawable
from cogent3.util.union_dict import UnionDict
from kath_analysis.fxy.transfer_annotations import (
    load_annotated,
    annotate_aligned,
)

import pandas as pd

In [2]:
def wrapper(names, name):
    name_index = names.index(name)
    def foo(data):
        return data[name_index] != "-"
    return foo 


In [3]:

# loading an annotated sequence which was saved as an alignment
s = load_aligned_seqs("/Users/katherine/repos/data/fxy/introns/aligned/annotated/intron3.json")

foo = wrapper(s.names, "Mmus")
mus_degapped_aln = s.take_positions_if(foo)

mus_degapped_aln.write("introns/3-degapped.fa")

intron_3 = pd.read_csv('~/repos/results/aim_3/rodent/mmu/without_gaps/intron_3/delta-conv-nogaps.tsv', sep="\t")

In [4]:
anno_seqs = load_annotated(
[ "~/repos/data/fxy/Mmus/annotated_simp_seq.json"])

anno_aln = annotate_aligned(
    "introns/3-degapped.fa",
    anno_seqs,
)
anno_aln.write(
    "introns/3-annotated.json"
)

In [5]:
s = load_aligned_seqs("introns/3-annotated.json")

In [6]:
le_sorted = intron_3.sort_values(by="pos")

In [7]:
# create a cogent3 Drawable
draw = Drawable(
    xtitle="Position",
    ytitle="delta_nabla",
)

In [8]:
data = []
last_index = 0 

for index, row in le_sorted.iterrows():
    if row['pos']-last_index > 100:
        
        df = pd.DataFrame(data, columns=['pos','d-conv'])
        scatt = UnionDict(
            type="scatter",
            x=df['pos'].values,
            y=df['d-conv'].values,
            mode="lines",
            name="δ<sub>nabla</sub>",
            line=dict(shape="spline", smoothing=1.3, color="cornflowerblue"),
            legendgroup = 'a', showlegend=False,
            )
        draw.traces.extend([scatt])
        data = []
        data.append((row['pos'], row['d-conv']))
        last_index = row['pos']
    else:
        data.append((row['pos'], row['d-conv']))
        last_index = row['pos']
df = pd.DataFrame(data, columns=['pos','d-conv'])
scatt = UnionDict(
    type="scatter",
    x=df['pos'].values,
    y=df['d-conv'].values,
    mode="lines",
    name="δ<sub>nabla</sub>",
    line=dict(shape="spline", smoothing=1.3, color="cornflowerblue"),
    legendgroup = 'a', showlegend=False,
    )
draw.traces.extend([scatt])

In [9]:

# get the annotations as a drawable
annots = s.get_drawable(width=600)
# the AnnotatedDrawable allows you to specify a simple subplot!
draw = AnnotatedDrawable(
    draw,
    bottom_track=annots,
    ytitle="$\Large \hat \delta_\\nabla$"
)
draw.layout.yaxis["domain"] = [0.3135, 1.0]
draw.layout["yaxis2"] = {"domain": [0.0, 0.2985]}


In [14]:
draw.layout['showlegend']=False

In [15]:
draw.write("/Users/katherine/repos/Thesis-Draft-1-KC/figures/plots/rodent/dconv-intron3.jpg")
draw.write("/Users/katherine/repos/Thesis-Draft-1-KC/figures/plots/rodent/dconv-intron3.pdf")